# caller

> Interact with CanLii programmatically

In [ ]:
#| default_exp caller

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import requests
import json

In [ ]:
#| export
class Caller:
    "Enter your APIkey and your preferred language ('en' or 'fr')"
    def __init__(self, key, language):
        self.key = key
        self.language = language
    
    def isolate_id(self, dictionary):
        #used for the list_decisions call
        #some databases return caseId as a language
        try:
            case_id = dictionary[self.language]
        except:
            if self.language == 'en':
                case_id = dictionary['fr']
            else:
                case_id = dictionary['en']
        return case_id    
        
    def list_tribunals(self):
        "No arguments required. Automatically checks CanLii for the latest list of databases."
        res = requests.get(f'https://api.canlii.org/v1/caseBrowse/{self.language}/?api_key={self.key}')
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['caseDatabases'])
        return df
    
    def list_decisions(self, databaseId, offset=0, resultCount=100):
        "Offset: (0 = descending, 1 = ascending)"
        res = requests.get(f'https://api.canlii.org/v1/caseBrowse/{self.language}/{databaseId}/?offset={offset}&resultCount={resultCount}&api_key={self.key}')
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['cases'])
        df['caseId'] = df['caseId'].apply(self.isolate_id)
        df.drop(columns=['databaseId'], inplace=True)
        return df
    
    def case_metadata(self, databaseId, caseId):
        res = requests.get(f'https://api.canlii.org/v1/caseBrowse/{self.language}/{databaseId}/{caseId}/?api_key={self.key}')
        json_res = json.loads(res.text)
        df = pd.Series(json_res)
        return df
    
    def case_cites_of_cases(self, databaseId, caseId):
        "Get a list of all the cases your target case cites."
        res = requests.get(f'https://api.canlii.org/v1/caseCitator/en/{databaseId}/{caseId}/citedCases?api_key={self.key}')
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['citedCases'])
        df['caseId'] = df['caseId'].apply(self.isolate_id)
        df.drop(columns=['databaseId'], inplace=True)
        return df
    
    def case_cites_of_legislation(self, databaseId, caseId):
        "Get a list of all the legislation your target case cites."
        res = requests.get(f'https://api.canlii.org/v1/caseCitator/en/{databaseId}/{caseId}/citedLegislations?api_key={self.key}')
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['citedLegislations'])
        return df
    
    def cites_of_case(self, databaseId, caseId):
        "Get a list of all the cases that cite your target case."
        res = requests.get(f'https://api.canlii.org/v1/caseCitator/en/{databaseId}/{caseId}/citingCases?api_key={self.key}')
        json_res = json.loads(res.text)
        df = pd.DataFrame.from_dict(json_res['citingCases'])
        df['caseId'] = df['caseId'].apply(self.isolate_id)
        df.drop(columns=['databaseId'], inplace=True)
        return df

## Instantiate the object
Enter your API and preferred language when you substantiate the Caller.
```sh
API = "YOUR SECRET KEY"
language = 'en'

my_caller = Caller(API,language)
```

## Use the Caller to make discrete CanLii requests
The caller can:

1. List all CanLii tribunal databases
2. List all decisions in a CanLii database
3. Query metadata for a specific case
4. List the cases the target case cites
5. List the legislation the target case cites
6. List the cases that cite the target case

In [ ]:
show_doc(Caller.list_tribunals)

---

[source](https://github.com/simon-lawyer/call-canlii/blob/master/call_canlii/caller.py#L30){target="_blank" style="float:right; font-size:smaller"}

### Caller.list_tribunals

>      Caller.list_tribunals ()

No arguments required. Automatically checks CanLii for the latest list of databases.

In [ ]:
show_doc(Caller.list_decisions)

---

[source](https://github.com/simon-lawyer/call-canlii/blob/master/call_canlii/caller.py#L37){target="_blank" style="float:right; font-size:smaller"}

### Caller.list_decisions

>      Caller.list_decisions (databaseId, offset=0, resultCount=100)

Offset: (0 = descending, 1 = ascending)

In [ ]:
show_doc(Caller.case_metadata)

---

[source](https://github.com/simon-lawyer/call-canlii/blob/master/call_canlii/caller.py#L46){target="_blank" style="float:right; font-size:smaller"}

### Caller.case_metadata

>      Caller.case_metadata (databaseId, caseId)

In [ ]:
show_doc(Caller.case_cites_of_cases)

---

[source](https://github.com/simon-lawyer/call-canlii/blob/master/call_canlii/caller.py#L52){target="_blank" style="float:right; font-size:smaller"}

### Caller.case_cites_of_cases

>      Caller.case_cites_of_cases (databaseId, caseId)

Get a list of all the cases your target case cites.

In [ ]:
show_doc(Caller.case_cites_of_legislation)

---

[source](https://github.com/simon-lawyer/call-canlii/blob/master/call_canlii/caller.py#L61){target="_blank" style="float:right; font-size:smaller"}

### Caller.case_cites_of_legislation

>      Caller.case_cites_of_legislation (databaseId, caseId)

Get a list of all the legislation your target case cites.

In [ ]:
show_doc(Caller.cites_of_case)

---

[source](https://github.com/simon-lawyer/call-canlii/blob/master/call_canlii/caller.py#L68){target="_blank" style="float:right; font-size:smaller"}

### Caller.cites_of_case

>      Caller.cites_of_case (databaseId, caseId)

Get a list of all the cases that cite your target case.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()